# 期末考察内容 2：DNA 序列特征提取

导入包，预设参数。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.cluster.hierarchy as sch

plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False
plt.tight_layout()

读取 DNA 序列数据集，并清洗数据。

In [ ]:
ELEMENT_TYPES = ["a", "t", "c", "g"]
elements = pd.DataFrame(columns=ELEMENT_TYPES)

with open("data/Art-model-data.txt", "r", encoding="utf-8") as fr:
    record_num = 0

    for index, line in enumerate(fr.readlines()):
        if index == 0 or len(line) < 20:
            continue

        elements.loc[record_num] = [
            line.count(element_type) for element_type in ELEMENT_TYPES
        ]
        record_num += 1

frequencies = elements.div(elements.sum(axis=1), axis=0)

a_frequencies = frequencies[:10]
b_frequencies = frequencies[10:]

为两种类型的序列分别绘制成分组成的饼图。

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

a_frequencies.sum().plot(
    kind="pie",
    title="A类序列组成饼图",
    autopct="%1.1f%%",
    ax=axes[0],
)
b_frequencies.sum().plot(
    kind="pie",
    title="B类序列组成饼图",
    autopct="%1.1f%%",
    ax=axes[1],
)

plt.show()

绘制两个序列中各个字符的平均出现频率。

In [ ]:
mean = pd.concat([a_frequencies.mean(), b_frequencies.mean()], axis=1)
mean.columns = ["A类", "B类"]
mean.plot(kind="bar")
plt.title("序列中各字符的平均频率")
plt.show()

绘制各序列内字符频率的柱状图。

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 8))

a_frequencies.plot(kind="bar", ax=axes[0])
b_frequencies.plot(kind="bar", ax=axes[1])

plt.suptitle("各序列内字符频率柱状图")
plt.legend("upper left")
plt.show()

对相似的序列进行聚类。

In [ ]:
plt.figure(figsize=(16, 8))
Z = sch.linkage(frequencies, method="ward", metric="euclidean")
sch.dendrogram(Z, labels=frequencies.index + 1, orientation="left")
plt.title("相似序列聚类图")
plt.show()

输出每个类内的序列。

In [ ]:
Z = sch.linkage(frequencies, method="ward", metric="euclidean")
f = sch.fcluster(Z, t=2, criterion="maxclust")
result = pd.DataFrame(data=f, columns=["类别"], index=frequencies.index)

for i in range(2):
    print(f"第{i + 1}类：{', '.join([str(k + 1) for k in list(result[result['类别'] == i + 1].index)])}")